In [1]:
# -*- coding: utf-8 -*-
"""
Created on Tue May 19 13:08:57 2020

@author: sujay
"""

import numpy as np
import matplotlib as mp
import matplotlib.pyplot as plt
import matplotlib.animation as animation

import IPython

def x_next(x,u,evolve,millis,noise):
    
    if millis :
        k_v = 100 
    else :
        k_v = 1 
    
    if noise :
        n = np.random.rand(1)*np.array([1,1])/100
    else :
        n = np.array([0,0])
    n = np.hstack( (n,np.array([0,0])) )
    
    x_temp = np.zeros((4,11))
    x_temp[:,0] = x.copy()

    dt = 0.01
    dt_internal = 0.001
    iterator = int(dt/dt_internal)

    for i in range(iterator):
        B = np.zeros((2,2))
        B[0,:] = np.array( [1.666+np.cos(x_temp[1,i]),0.333+0.5*np.cos(x_temp[1,i])] )
        B[1,:] = np.array( [0.333+0.5*np.cos(x_temp[1,i]),0.333] )
        temp_var_1 = np.zeros((2,1))
        temp_var_1[0,0] = u[0] + np.sin(x_temp[1,i])*( x_temp[2,i]*x_temp[3,i] + 0.5*(x_temp[3,i]**2) ) - k_v*x_temp[2,i]
        temp_var_1[1,0] = u[1] - 0.5*np.sin(x_temp[1,i])*(x_temp[2,i]**2) - k_v*x_temp[3,i]
        theta_dot_dot = np.dot(np.linalg.inv(B),temp_var_1)
        temp_var_2 = np.array( [x_temp[2,i],x_temp[3,i]] )
        f = np.hstack((temp_var_2,theta_dot_dot[:,0]))+n
        x_temp[:,i+1] = x_temp[:,i] + f*dt_internal

    if evolve :
        result_temp = x_temp[:,1:11]
    else :
        result_temp = x_temp[:,10]

    if millis :
        result_temp = x_temp[:,1]

    return result_temp

def f(x_guess,u_guess):
    B = np.zeros((2,2))
    B[0,:] = np.array( [1.666+np.cos(x_guess[1]),0.333+0.5*np.cos(x_guess[1])] )
    B[1,:] = np.array( [0.333+0.5*np.cos(x_guess[1]),0.333] )
    temp_var_1 = np.zeros((2,1))
    temp_var_1[0,:] = u_guess[0] + np.sin(x_guess[1])*( x_guess[2]*x_guess[3] + 0.5*(x_guess[3]**2) ) - x_guess[2]
    temp_var_1[1,:] = u_guess[1] - 0.5*np.sin(x_guess[1])*(x_guess[2]**2) - x_guess[3]
    theta_dot_dot = np.dot(np.linalg.inv(B),temp_var_1)
    temp_var_2 = np.array( [x_guess[2],x_guess[3]] )
    result = np.hstack((temp_var_2,theta_dot_dot[:,0]))
    return result

def del_f_x(x_guess,u_guess):
    
    f32 = ( (500000*np.sin(x_guess[1])*((np.sin(x_guess[1])*x_guess[2]**2)/2 - u_guess[1] + x_guess[3])) - (333000*np.cos(x_guess[1])*(x_guess[3]**2/2 + x_guess[2]*x_guess[3])) - (500*x_guess[2]**2*np.cos(x_guess[1])*(500*np.cos(x_guess[1]) + 333)) )/(250000*np.cos(x_guess[1])**2 - 443889) + ( - (166500000000*np.cos(x_guess[1])*np.sin(x_guess[1])*(u_guess[0] - x_guess[2] + np.sin(x_guess[1])*(x_guess[3]**2/2 + x_guess[2]*x_guess[3]))) - (500000000*np.cos(x_guess[1])*np.sin(x_guess[1])*(500*np.cos(x_guess[1]) + 333)*((np.sin(x_guess[1])*x_guess[2]**2)/2 - u_guess[1] + x_guess[3])) )/(250000*np.cos(x_guess[1])**2 - 443889)**2 
    f33 = (- (333000*(x_guess[3]*np.sin(x_guess[1]) - 1)) - (1000*x_guess[2]*np.sin(x_guess[1])*(500*np.cos(x_guess[1]) + 333)))/(250000*np.cos(x_guess[1])**2 - 443889) 
    f34 = (- (1000*(500*np.cos(x_guess[1]) + 333)) - (333000*np.sin(x_guess[1])*(x_guess[2] + x_guess[3])))/(250000*np.cos(x_guess[1])**2 - 443889) 
    
    f42 = ( (1000*x_guess[2]**2*np.cos(x_guess[1])*(500*np.cos(x_guess[1]) + 833)) - (500000*np.sin(x_guess[1])*(u_guess[0] - x_guess[2] + np.sin(x_guess[1])*(x_guess[3]**2/2 + x_guess[2]*x_guess[3]))) - (1000000*np.sin(x_guess[1])*((np.sin(x_guess[1])*x_guess[2]**2)/2 - u_guess[1] + x_guess[3])) + (1000*np.cos(x_guess[1])*(500*np.cos(x_guess[1]) + 333)*(x_guess[3]**2/2 + x_guess[2]*x_guess[3])) )/(250000*np.cos(x_guess[1])**2 - 443889) + ( (1000000000*np.cos(x_guess[1])*np.sin(x_guess[1])*(500*np.cos(x_guess[1]) + 833)*((np.sin(x_guess[1])*x_guess[2]**2)/2 - u_guess[1] + x_guess[3])) + (500000000*np.cos(x_guess[1])*np.sin(x_guess[1])*(500*np.cos(x_guess[1]) + 333)*(u_guess[0] - x_guess[2] + np.sin(x_guess[1])*(x_guess[3]**2/2 + x_guess[2]*x_guess[3]))) )/(250000*np.cos(x_guess[1])**2 - 443889)**2 
    f43 = ( (1000*(x_guess[3]*np.sin(x_guess[1]) - 1)*(500*np.cos(x_guess[1]) + 333)) + (2000*x_guess[2]*np.sin(x_guess[1])*(500*np.cos(x_guess[1]) + 833)) )/(250000*np.cos(x_guess[1])**2 - 443889) 
    f44 = ( (2000*(500*np.cos(x_guess[1]) + 833)) + (1000*np.sin(x_guess[1])*(x_guess[2] + x_guess[3])*(500*np.cos(x_guess[1]) + 333)) )/(250000*np.cos(x_guess[1])**2 - 443889) 
    
    result = np.zeros((4,4))
    result[0,:] = np.array( [0,0,1,0] )
    result[1,:] = np.array( [0,0,0,1] )
    result[2,:] = np.array( [0,f32,f33,f34] )
    result[3,:] = np.array( [0,f42,f43,f44] )
    
    return result

def del_f_u(x_guess):
    
    f31 = -333000/(250000*np.cos(x_guess[1])**2 - 443889)  
    f32 = (1000*(500*np.cos(x_guess[1]) + 333))/(250000*np.cos(x_guess[1])**2 - 443889)  
    f41 = (1000*(500*np.cos(x_guess[1]) + 333))/(250000*np.cos(x_guess[1])**2 - 443889)  
    f42 = -(2000*(500*np.cos(x_guess[1]) + 833))/(250000*np.cos(x_guess[1])**2 - 443889)
    
    result = np.zeros((4,2))
    result[0,:] = np.array( [0,0] )
    result[1,:] = np.array( [0,0] )
    result[2,:] = np.array( [f31,f32] )
    result[3,:] = np.array( [f41,f42] )

    return result

def iLQR_controller(horizon_length,x_0,x_d,iterator):
    
    dt = 0.01 
    Q =  np.zeros((4,4))
    Q[0,:] = np.array([1,0,0,0])
    Q[1,:] = np.array([0,1,0,0])
    Q = 50000*Q
    R = 1*np.eye(2)

    u_guess = np.zeros((2,horizon_length-1))
    x_guess = np.zeros((4,horizon_length))
    x_cal = np.zeros((4,horizon_length))
    x_guess[:,0] = x_0.copy()
    x_cal[:,0] = x_0.copy()
    for i in range(1,horizon_length):
        x_guess[:,i] = x_next(x_guess[:,i-1],u_guess[:,i-1],False,False,False)
        
    K_temp = np.zeros((horizon_length-1,2,4)) 
    k_temp = np.zeros((horizon_length-1,2,1)) 
    S = np.zeros((horizon_length,4,4))
    M = np.zeros((horizon_length,4,1))
    S[-1] = Q.copy()
    M[-1] = -np.dot(Q,np.transpose([x_d]))

    for i1 in range(iterator):
        
        for i in range(horizon_length-2,-1,-1):
            A = np.eye(4) + del_f_x(x_guess[:,i],u_guess[:,i])*dt
            B = del_f_u(x_guess[:,i])*dt
            
            C = ( np.transpose([f(x_guess[:,i],u_guess[:,i])]) - np.dot(del_f_x(x_guess[:,i],u_guess[:,i]),np.transpose([x_guess[:,i]])) - np.dot(del_f_u(x_guess[:,i]),np.transpose([u_guess[:,i]])) )*dt
            temp_var_1 = R + np.dot(np.transpose(B),np.dot(S[i+1],B))
            temp_var_1 = - np.linalg.inv(temp_var_1)
            temp_var_2 = np.dot(S[i+1],A)
            temp_var_3 = np.dot(np.transpose(B),temp_var_2)
            K_temp[i] = np.dot(temp_var_1,temp_var_3)
            temp_var_4 = np.dot(S[i+1],C) + M[i+1]
            k_temp[i] = np.dot(temp_var_1,np.dot(np.transpose(B),temp_var_4))
            temp_var_5 = np.dot(np.transpose(temp_var_3),np.dot(temp_var_1,temp_var_3))
            S[i] = Q + np.dot(np.transpose(A),temp_var_2) + temp_var_5
            temp_var_6 = np.transpose(A) + np.dot(np.transpose(temp_var_3),np.dot(temp_var_1,np.transpose(B)))
            M[i] = -np.dot(Q,np.transpose([x_d])) + np.dot(temp_var_6,temp_var_4)
            
        for i in range(horizon_length-1):
            u_guess_temp = np.dot(K_temp[i],np.transpose([x_cal[:,i]])) +  k_temp[i]
            u_guess[:,i] = u_guess_temp[:,0]
            x_cal[:,i+1] = x_next(x_guess[:,i],u_guess[:,i],False,False,False)

        x_guess = x_cal.copy()
    
    return K_temp[0],k_temp[0]    

l_arm = 0.5

"""
Generate an random punch
"""
Angle_punch = np.random.randint(91,100) 
Distance_punch = np.random.randint(50,75)/100 
Velocity_punch = np.random.randint(40,50)/10 

#Generate a random interception point
x_y_d = np.array([-np.random.randint(10,20)/100,np.random.randint(50,60)/100])

theta_2_d = np.arccos((x_y_d[0]**2+x_y_d[1]**2)/(2*l_arm**2) - 1)

s1 = (1+np.cos(theta_2_d))*x_y_d[1] - np.sin(theta_2_d)*x_y_d[0]
c1 = (1+np.cos(theta_2_d))*x_y_d[0] + np.sin(theta_2_d)*x_y_d[1]

theta_1_d = np.arctan2(s1,c1)

#Calculate the desired destination
x_d = np.array([theta_1_d,theta_2_d,0,0])

#Calculate the time to intercept and generate corresponding horizon length
time_intercept = Distance_punch/Velocity_punch ;
horizon_length = int((time_intercept - time_intercept%0.01)*100)

#initialize the starting point
x_0 = np.array([np.pi/4,3*np.pi/4,0,0])

"""
MPC with iLQR
"""

x_real = np.zeros([4,1])
x_real[:,0] = x_0.copy()
u_real = np.zeros((2,horizon_length-1))
#Extend the horizon to witness the interception
ext_horizon_length = horizon_length + 25
for i in range(ext_horizon_length-1):
    
    if i > ( horizon_length - 2 ):
        u_real_temp = np.array([0,0])
    else :
        K , k = iLQR_controller(horizon_length - i,x_real[:,-1],x_d,10)
        u_real_temp = np.dot(K,np.transpose([x_real[:,-1]])) + k
        u_real_temp = u_real_temp[:,0]
        u_real[:,i] = u_real_temp.copy()
        
    x_temp = x_next(x_real[:,-1],u_real_temp,True,False,True)
    x_real = np.hstack((x_real, x_temp))

#plot output
    
intercept_flag = False

x_y_arm = np.zeros((len(x_real[0,:]),2)) 
x_y_joint = np.zeros((len(x_real[0,:]),2)) 
x_y_punch = np.zeros((len(x_real[0,:]),2)) 

x_y_punch[0] = Distance_punch*np.array([np.cos(np.deg2rad(Angle_punch)),np.sin(np.deg2rad(Angle_punch))])+x_y_d

for i in range(len(x_real[0,:])):
    x_y_arm[i] = np.array([ 0.5*(np.cos(x_real[0,i])+np.cos(x_real[0,i]+x_real[1,i])),0.5*(np.sin(x_real[0,i])+np.sin(x_real[0,i]+x_real[1,i])) ])
    x_y_joint[i] = np.array([ 0.5*np.cos(x_real[0,i]),0.5*np.sin(x_real[0,i])])
    if i != 0 :
        if intercept_flag :
            x_y_punch[i] = x_y_arm[i]
        else:
            x_y_punch_temp = x_y_punch[i-1] - 0.001*Velocity_punch*np.array([np.cos(np.deg2rad(Angle_punch)),np.sin(np.deg2rad(Angle_punch))])
            x_y_punch[i] = x_y_punch_temp
            if (np.linalg.norm(x_y_punch_temp - x_y_arm[i]) < 0.05) :
                intercept_flag = True      

use_dt = 5

fig = mp.figure.Figure(figsize=[9,12])
mp.backends.backend_agg.FigureCanvasAgg(fig)
ax = fig.add_subplot(111, autoscale_on=False, xlim=[-1,0.5], ylim=[-0.5,2])
ax.grid()
    
#plot the ground
ax.plot([-2,2],  [-0.1,-0.1],'c')
#plot the body
ax.plot([-0.5,0],  [0,0],'k', lw=4)
ax.plot([-0.5,-0.5],  [-0.1,0],'k', lw=4)
ax.plot([0,0],  [-0.1,0],'k', lw=4)

#Create two planar arm
list_of_lines = []
#for the link 1
line, = ax.plot([], [], 'k', lw=2)
list_of_lines.append(line)
#for the link 2
line, = ax.plot([], [], 'k', lw=2)
list_of_lines.append(line)
#for the punch
line, = ax.plot([], [], 'oc', ms=30)
list_of_lines.append(line)

def animate(i):

    for l in list_of_lines: #reset all lines
        l.set_data([],[])
                        
    list_of_lines[0].set_data([0, x_y_joint[i,0]], [0, x_y_joint[i,1]])
    list_of_lines[1].set_data([x_y_joint[i,0], x_y_arm[i,0]], [x_y_joint[i,1], x_y_arm[i,1]])
    list_of_lines[2].set_data([x_y_punch[i,0], x_y_punch[i,0]], [x_y_punch[i,1], x_y_punch[i,1]])
        
    return list_of_lines
    
def init():
    return animate(0)

ani = animation.FuncAnimation(fig, animate, np.arange(0, len(x_real[0,:])),interval=use_dt, blit=True, init_func=init)

plt.close(fig)
plt.close(ani._fig)
IPython.display.display_html(IPython.core.display.HTML(ani.to_html5_video()))

"""
Consider running the code again if the punch is not intercepted
"""

t = np.arange(0, len(x_real[0,:]))*0.001;
t = np.arange(0, len(u_real[0,:]))*0.01;
'''
plt.plot(t,x_real[0,:])
plt.ylabel('theta 1')
plt.xlabel('Time')

plt.plot(t,x_real[1,:])
plt.ylabel('theta 2')
plt.xlabel('Time')

plt.plot(t,x_real[2,:])
plt.ylabel('omg 1')
plt.xlabel('Time')

plt.plot(t,x_real[3,:])
plt.ylabel('omg 2')
plt.xlabel('Time')

plt.plot(t,u_real[0,:])
plt.ylabel('u 1')
plt.xlabel('Time')

plt.plot(t,u_real[1,:])
plt.ylabel('u 2')
plt.xlabel('Time')
'''

Your browser does not support the video tag.

"\nplt.plot(t,x_real[0,:])\nplt.ylabel('theta 1')\nplt.xlabel('Time')\n\nplt.plot(t,x_real[1,:])\nplt.ylabel('theta 2')\nplt.xlabel('Time')\n\nplt.plot(t,x_real[2,:])\nplt.ylabel('omg 1')\nplt.xlabel('Time')\n\nplt.plot(t,x_real[3,:])\nplt.ylabel('omg 2')\nplt.xlabel('Time')\n\nplt.plot(t,u_real[0,:])\nplt.ylabel('u 1')\nplt.xlabel('Time')\n\nplt.plot(t,u_real[1,:])\nplt.ylabel('u 2')\nplt.xlabel('Time')\n"